In [4]:
%run setup_levy_model_sternberg.py

/home/slesche/.local/lib/python3.10/site-packages/bayesflow/trainers.py:27: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
INFO:root:Performing 2 pilot runs with the lfm model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 5)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 100, 2)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:No optional simulation non-batchable context provided.
INFO:root:No optional simulation batchable context provided.


2.15.1
[[1.1 3.  0.3 0.2 1.5]] [[0.6 3.  0.2 0.2 0.3]]


INFO:root:Loaded loss history from checkpoints//sternberg/history_100.pkl.
INFO:root:Networks loaded from checkpoints//sternberg/ckpt-100
INFO:root:Performing a consistency check with provided components...
INFO:root:Done.


In [3]:
# Loading data
import pandas as pd
import os
import fnmatch
import glob 
from keras.utils import to_categorical

2024-06-13 09:06:38.503893: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-13 09:06:38.505379: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-13 09:06:38.527596: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 09:06:38.527615: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 09:06:38.528171: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def config_observed_data(forward_dict):
    """Configure the output of the GenerativeModel for a BayesFlow setup."""

    # Prepare placeholder dict
    out_dict = {}

    # Extract simulated response times
    data = forward_dict["data"]
    data = data[np.newaxis,:,:]
    out_dict["summary_conditions"] = data.astype(np.float32)

    # Make inference network aware of varying numbers of trials
    # We create a vector of shape (batch_size, 1) by repeating the sqrt(num_obs)
    vec_num_obs = np.sqrt(data.shape[0]) * np.ones((data.shape[0], 1))
    out_dict["direct_conditions"] = np.sqrt(vec_num_obs).astype(np.float32)

    return out_dict

In [5]:
def get_posterior_means(file):
    raw_data = pd.read_csv(file)
    # raw_data["stim"] = raw_data["stim"]-1
    # raw_data["rt"] = raw_data["rt"]/1000
    raw_data["rt"] = (raw_data["resp"]*2 - 1) * raw_data["rt"] # to negate wrong rts
    condition = raw_data["resp"]*0 # set it all to 0
    data = np.c_[condition, raw_data["rt"].to_numpy()]
    input_dict = {
        'data': data
    }
    
    theta_hat = amortizer.sample(config_observed_data(input_dict), 5000, to_numpy=True)
    std_means = np.mean(theta_hat, axis=0)
    return(prior_means + std_means*prior_stds)

In [8]:
# Loop through each datafile and estimate 
path_to_data = 'data/levy_data/sternberg'
PAR = pd.DataFrame(columns=['sub', 'condition', 'a', 'v', 't', 'st', 'alpha'])

for condition in [1, 2, 3]:
    mask = path_to_data + '/condition_', str(condition), '/sternberg_data*.csv'
    print(mask)
    datafiles = glob.glob(mask) 

    for file in datafiles:
        print(file)
        par = get_posterior_means(file)
        print(np.round(par,2))
        result = pd.DataFrame(par.reshape(1,7),columns=['a', 'v', 't', 'st', 'alpha'])
        result['sub'] = file.split('_')[-1].split('.')[0]
        result['condition'] = condition
        PAR = pd.concat([PAR, result], ignore_index=True)    

('data/levy_data/sternberg/condition_', '1', '/sternberg_data*.csv')


TypeError: expected str, bytes or os.PathLike object, not tuple

In [15]:
PAR.to_csv(path_to_data + '/levy_mat_posner.csv', index=False)